<a href="https://colab.research.google.com/github/du-hr/MNIST-CNN/blob/good_copy/MiniProject_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>